In [1]:
%cd ../../../../

/home/users/dmoreno2016/ASTROMER/astromer_pe/astromer


In [16]:
import tensorflow as tf
import pandas as pd
import numpy as np

import os

from src.data.record import deserialize
from src.layers.positional import PositionalEmbedding

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
path_dataset = './data/records/alcock/fold_0/alcock_20/train'

rec_paths = []
for folder in os.listdir(path_dataset):
    if folder.endswith('.csv'):
        continue
    for x in os.listdir(os.path.join(path_dataset, folder)):
        rec_paths.append(os.path.join(path_dataset, folder, x))

dataset = tf.data.TFRecordDataset(rec_paths)    
dataset = dataset.map(deserialize)

In [35]:
for lc_info in dataset:
    lc_data = lc_info['input']
    break

lc_data.shape

TensorShape([635, 3])

In [42]:
toy_lc_data = lc_data[0:7,:]
toy_lc_data

<tf.Tensor: shape=(7, 3), dtype=float32, numpy=
array([[ 4.8917543e+04, -5.0609999e+00,  3.7999999e-02],
       [ 4.8918762e+04, -5.0390000e+00,  3.4000002e-02],
       [ 4.8919555e+04, -5.0730000e+00,  8.3999999e-02],
       [ 4.8920539e+04, -5.1550002e+00,  2.8999999e-02],
       [ 4.8927508e+04, -5.3639998e+00,  6.1999999e-02],
       [ 4.8928543e+04, -5.0679998e+00,  4.3000001e-02],
       [ 4.8929488e+04, -4.9740000e+00,  6.4000003e-02]], dtype=float32)>

In [45]:
toy_lc_time = toy_lc_data[:,0]
toy_lc_flux = toy_lc_data[:,1]

toy_lc_time = tf.expand_dims(tf.expand_dims(toy_lc_time, 1), 0)
toy_lc_flux = tf.expand_dims(tf.expand_dims(toy_lc_flux, 1), 0)

toy_lc_time

<tf.Tensor: shape=(1, 7, 1), dtype=float32, numpy=
array([[[48917.543],
        [48918.76 ],
        [48919.555],
        [48920.54 ],
        [48927.508],
        [48928.543],
        [48929.49 ]]], dtype=float32)>

In [51]:
import six
import math

from typing import Optional

Initializer = tf.keras.initializers.Initializer

def assert_rank(tensor, expected_rank, name=None):
  """Raises an exception if the tensor rank is not of the expected rank.

  Args:
    tensor: A tf.Tensor to check the rank of.
    expected_rank: Python integer or list of integers, expected rank.
    name: Optional name of the tensor for the error message.

  Raises:
    ValueError: If the expected shape doesn't match the actual shape.
  """
  expected_rank_dict = {}
  if isinstance(expected_rank, six.integer_types):
    expected_rank_dict[expected_rank] = True
  else:
    for x in expected_rank:
      expected_rank_dict[x] = True

  actual_rank = tensor.shape.ndims
  if actual_rank not in expected_rank_dict:
    raise ValueError(
        "For the tensor `%s`, the actual tensor rank `%d` (shape = %s) is not "
        "equal to the expected tensor rank `%s`" %
        (name, actual_rank, str(tensor.shape), str(expected_rank)))

def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.

  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.

  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if expected_rank is not None:
    assert_rank(tensor, expected_rank, name)

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape

def _relative_position_bucket(relative_position,
                              bidirectional=True,
                              num_buckets=32,
                              max_distance=128):
  """Translate relative position to a bucket number for relative attention.

  The relative position is defined as memory_position - query_position, i.e.
  the distance in tokens from the attending position to the attended-to
  position.

  If `bidirectional=False`, then positive relative positions are invalid.

  We use smaller buckets for small absolute relative_position and larger
  buckets for larger absolute relative_positions.

  All relative positions >=max_distance map to the same bucket.

  All relative positions <=-max_distance map to the same bucket.

  This should allow for more graceful generalization to longer sequences
  than the model has been trained on.

  Args:
    relative_position: An int32 Tensor
    bidirectional: A boolean - whether the attention is bidirectional
    num_buckets: An integer
    max_distance: An integer

  Returns:
    A Tensor with the same shape as relative_position, containing int32
    values in the range [0, num_buckets)
  """
  ret = 0
  n = -relative_position
  if bidirectional:
    num_buckets //= 2
    ret += tf.cast(tf.math.less(n, 0), tf.int32) * num_buckets
    n = tf.math.abs(n)
  else:
    n = tf.math.maximum(n, 0)
  # now n is in the range [0, inf)
  max_exact = num_buckets // 2
  is_small = tf.math.less(n, max_exact)
  val_if_large = max_exact + tf.dtypes.cast(
      tf.math.log(tf.cast(n, tf.float32) / max_exact) /
      math.log(max_distance / max_exact) * (num_buckets - max_exact),
      tf.int32,
  )
  val_if_large = tf.math.minimum(val_if_large, num_buckets - 1)
  ret += tf.where(is_small, n, val_if_large)
  return ret


@tf.keras.utils.register_keras_serializable(package="Text")
class RelativePositionBias(tf.keras.layers.Layer):
  """Relative position embedding via per-head bias in T5 style.

  Reference implementation in MeshTF:
  https://github.com/tensorflow/mesh/blob/master/mesh_tensorflow/transformer/transformer_layers.py#L1000

  This layer implements the relative position bias used in "Exploring the Limits
  of Transfer Learning with a Unified Text-to-Text Transformer"
  (https://arxiv.org/abs/1910.10683)
  """

  def __init__(self,
               num_heads: int,
               relative_attention_num_buckets: int = 32,
               relative_attention_max_distance: int = 128,
               bidirectional: bool = True,
               embeddings_initializer: Optional[Initializer] = None,
               **kwargs):
    super().__init__(**kwargs)
    self.num_heads = num_heads
    self.relative_attention_num_buckets = relative_attention_num_buckets
    self.bidirectional = bidirectional
    self.relative_attention_max_distance = relative_attention_max_distance
    if embeddings_initializer:
      self._embed_init = embeddings_initializer
    else:
      self._embed_init = tf.keras.initializers.TruncatedNormal(stddev=1.0)
    with tf.name_scope(self.name):
      self._relative_attention_bias = self.add_weight(
          "rel_embedding",
          shape=[self.relative_attention_num_buckets, self.num_heads],
          initializer=self._embed_init,
          dtype=self.dtype,
          trainable=True)

  def get_config(self):
    config = {
        "num_heads":
            self.num_heads,
        "relative_attention_num_buckets":
            self.relative_attention_num_buckets,
        "relative_attention_max_distance":
            self.relative_attention_max_distance,
        "bidirectional":
            self.bidirectional,
        "embeddings_initializer":
            tf.keras.initializers.serialize(self._embed_init),
    }
    base_config = super().get_config()
    return dict(list(base_config.items()) + list(config.items()))

  def call(self, query: tf.Tensor, key: tf.Tensor):
    """Implements the forward pass.

    Args:
      query: query input tensor shape [batch, query length, hidden size].
      key: key input tensor shape [batch, key length, hidden size].

    Returns:
      A tensor in shape of [batch, heads, query length, key length].
    """
    batch_size, qlen = get_shape_list(query)[:2]
    klen = get_shape_list(key)[1]
    context_position = tf.range(qlen)[:, None]
    memory_position = tf.range(klen)[None, :]
    relative_position = memory_position - context_position
    rp_bucket = _relative_position_bucket(
        relative_position,
        bidirectional=self.bidirectional,
        num_buckets=self.relative_attention_num_buckets,
        max_distance=self.relative_attention_max_distance)
    values = tf.nn.embedding_lookup(self._relative_attention_bias, rp_bucket)
    values = tf.expand_dims(
        tf.transpose(values, [2, 0, 1]),
        axis=0)  # shape (1, num_heads, qlen, klen)
    values = tf.tile(values, [batch_size, 1, 1, 1])
    return values

In [52]:
wq = tf.keras.layers.Dense(d_model, name='WQ')

query = wq(toy_lc_flux)
query

<tf.Tensor: shape=(1, 7, 6), dtype=float32, numpy=
array([[[ 0.13297616,  3.1281412 , -3.834384  , -0.81204396,
         -3.8657029 , -1.9403491 ],
        [ 0.13239811,  3.1145434 , -3.8177161 , -0.80851406,
         -3.848899  , -1.9319146 ],
        [ 0.13329145,  3.1355584 , -3.8434756 , -0.8139694 ,
         -3.8748689 , -1.9449499 ],
        [ 0.13544598,  3.1862416 , -3.9056017 , -0.82712644,
         -3.9375024 , -1.9763882 ],
        [ 0.14093739,  3.3154218 , -4.063947  , -0.86066073,
         -4.097141  , -2.0565171 ],
        [ 0.13316008,  3.1324677 , -3.8396873 , -0.81316715,
         -3.8710496 , -1.9430329 ],
        [ 0.13069026,  3.0743678 , -3.76847   , -0.79808474,
         -3.7992504 , -1.9069941 ]]], dtype=float32)>

In [53]:
get_shape_list(query)[:2]

[1, 7]